In [1]:
# Import packages
# Data manipulation
import re
import pickle
import math
import numpy as np
import pandas as pd
import nltk
#first time usage: download addtional packages form nltk first:
#nltk.download()
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util

/Users/ac/miniforge3/envs/sam/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Get sample lyrics set based on "well-known" artists
# In this csv we can identify A$AP Ferg and A-ha
df = pd.read_csv("./songsdata/songsdata_1500.csv")
df['artist'].unique()

array(['A Storm of Light', 'A Sunset Diary', 'Various Artists',
       'A Taste Of Honey', 'Guatauba', 'A Thorn For Every Heart',
       'A Thousand Times Repent', 'A Textbook Tragedy', 'A TiRO',
       'A Traitor Like Judas', 'A Tribe Called Quest',
       'Original Soundtrack', 'A Toys Orchestra',
       'A Trillion Barnacle Lapse', 'A Trunk Full of Dead Bodies',
       'A Vain Attempt', 'A Turma Do Balão Mágico', 'A Verse Unsung',
       'A Well Thought Tragedy', 'A Voice Like Rhetoric', 'A Weather',
       'A Week In July', 'A$AP Ferg', 'A Whisper in the Noise',
       'James Newton Howard', 'A Wilhelm Scream', 'A-bros', 'a-ha',
       'A$AP Rocky', 'Smoke DZA', 'Lloyd Banks', 'A-Wax',
       'A tirador láser'], dtype=object)

In [4]:
sample_artists_set = df[(df['artist']=='A$AP Ferg') | (df['artist']=='a-ha')].dropna().drop_duplicates()

### preprocessing ideas

In [5]:
# remove lyrics that are not english
from langdetect import detect

sample_artists_set['lang'] = sample_artists_set['lyrics'].apply(detect)
sample_artists_set = sample_artists_set[sample_artists_set['lang']=='en']
sample_artists_set

,artist,title,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration,time_signature,lyrics,lang
225,A$AP Ferg,Let It Go,0.785,0.778,9,-6.003,0,0.1650,0.2470,0.00000,0.6520,0.769,119.976,282640,4,Let It Go Lyrics[Intro: A$AP Yams]\nTrap Lord ...,en
226,A$AP Ferg,Shabba (feat. A$AP Rocky),0.869,0.712,9,-6.136,1,0.1030,0.0715,0.00000,0.2720,0.435,120.000,275867,4,This Is A$AP Ferg Lyrics1. A$AP Ferg- Move Ya ...,en
227,A$AP Ferg,Lord (feat. Bone Thugs-n-Harmony),0.730,0.890,7,-6.410,1,0.1680,0.4030,0.00000,0.6110,0.446,125.405,317187,4,Lord Lyrics[Produced by: Ozhora Miyagi & Cryst...,en
228,A$AP Ferg,Hood Pope,0.731,0.631,9,-7.006,1,0.1550,0.1060,0.00000,0.2200,0.513,131.977,210747,4,"Hood Pope Lyrics[Chorus]\nOhh, let me sing my ...",en
229,A$AP Ferg,Fergivicious,0.859,0.842,6,-5.170,0,0.1960,0.1710,0.00000,0.1650,0.610,125.981,230520,4,"Fergivicious Lyrics[Intro]\nYeah, a lot of you...",en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,a-ha,Sunny Mystery,0.612,0.637,9,-8.012,0,0.0251,0.0189,0.22600,0.1010,0.273,135.005,210627,4,Sunny Mystery LyricsYou can run through every ...,en
453,a-ha,Start The Simulator,0.491,0.633,9,-6.352,1,0.0249,0.4080,0.00967,0.1100,0.172,95.945,317160,3,Start the Simulator LyricsStart the simulator\...,en
454,a-ha,Case Closed On Silver Shore,0.655,0.707,11,-8.707,1,0.0290,0.1350,0.06960,0.0647,0.515,100.069,268973,4,Case Closed On Silver Shore LyricsHere's a sta...,en
455,a-ha,Lifelines - Demo,0.632,0.400,7,-9.238,1,0.0252,0.2000,0.00291,0.1150,0.174,77.997,316987,4,Lifelines (Demo) LyricsOne time to know that i...,en


In [6]:
lyrics_set = sample_artists_set['lyrics']
lyrics_set

225    Let It Go Lyrics[Intro: A$AP Yams]\nTrap Lord ...
226    This Is A$AP Ferg Lyrics1. A$AP Ferg- Move Ya ...
227    Lord Lyrics[Produced by: Ozhora Miyagi & Cryst...
228    Hood Pope Lyrics[Chorus]\nOhh, let me sing my ...
229    Fergivicious Lyrics[Intro]\nYeah, a lot of you...
                             ...                        
452    Sunny Mystery LyricsYou can run through every ...
453    Start the Simulator LyricsStart the simulator\...
454    Case Closed On Silver Shore LyricsHere's a sta...
455    Lifelines (Demo) LyricsOne time to know that i...
456    Summer Moved On (Remix) LyricsSummer moved on\...
Name: lyrics, Length: 120, dtype: object

In [10]:
l_pickle = [sample_artists_set, lyrics_set]
with open('./pickle_objects/sample_song_lyrics_set.obj', 'wb') as f:
    pickle.dump(l_pickle, f)